# WeatherPy 
UCB Extension HW6
written by A. Lam

In [33]:
# Modules
# Other modules are imported within the functions in which they are relevant
import pandas as pd

In [31]:
# Sample API Query
# API query structure
base_url = 'http://api.openweathermap.org/data/2.5/weather'
api_key = '&appid=' + owm.avl
# by city id query string
city_id = 'tainan'
country_code = 'tw'
city_id_q = '?q=' + city_id + ',' + country_code
units = '&units=imperial'
url = base_url+city_id_q+units+api_key

# query to API
response = req.get(url).json()
# print(json.dumps(response,indent=2,sort_keys = True))

# Humidity (%)
# response['main']['humidity']
# Temperature (def = K, units = imperial: F)
# response['main']['temp']
# Temp. alternatives:
# response['main']['temp_max']
# response['main']['temp_min']
# Cloudiness (%)
# response['clouds']['all']
# Windspeed (def = m/s, units = imperial: mph)
# response['wind']['speed']

In [34]:
# Create function to generate coordinates
def rand_coord():
    import random
    # Lat ranges from degrees [-90, 90] (N,S)
    lat = round(180*random.random()-90,2) 
    # Long ranges from degrees [-180, 180] (E,W)
    long = round(360*random.random()-180,2)
    return [lat,long]

In [35]:
# Create function to convert coordinates to city and country code string
def coord2city(coords):
    # assume coords is a 2 item list of form: [lat,long]
    from citipy import citipy
    # extracts city name
    city = citipy.nearest_city(coords[0],coords[1]).city_name
    # extracts country code
    country = citipy.nearest_city(coords[0],coords[1]).country_code
    out = [city, country]
    return out

In [44]:
# Create function to automate API call for random city
def rand_city_data():
    # Modules
    import requests as req
    import json
    import OpenWeatherMap_API as owm
    import pandas as pd
    # generate city and log coords, name, and country
    coords = rand_coord()
    place = coord2city(coords)
    
    # Keep variables for output
    city = place[0]
    country_code = place[1]
    
    city_id_q = '?q=' + city + ',' + country_code
    # API query structure
    base_url = 'http://api.openweathermap.org/data/2.5/weather'
    api_key = '&appid=' + owm.avl
    units = '&units=imperial'
    # construct query URL
    url = base_url+city_id_q+units+api_key
    # make API call
    response = req.get(url).json()
    
    # output variables
    humidity = response['main']['humidity']
    temperature = response['main']['temp']
    cloudiness = response['clouds']['all']
    windspeed = response['wind']['speed']
    url_clean = base_url+city_id_q+units+'&appid=InsertKey'
    latitude = coords[0]
    longitude = coords[1]
    
    # Create dataframe of output variables
    output_order = ['City', 'Country Code', 'Latitude',
        'Longitude', 'Temperature', 'Humidity',
        'Cloudiness', 'Wind Speed', 'URL']
    out = pd.DataFrame({
        'City' : city,
        'Country Code' : country_code,
        'Latitude' : latitude,
        'Longitude' : longitude,
        'Temperature' : temperature,
        'Humidity' : humidity,
        'Cloudiness' : cloudiness,
        'Wind Speed' : windspeed,
        'URL' : url_clean
    }, index = [0])
    
    return out[output_order]

In [45]:
# Test functions to generate cities
test = rand_coord()
print(test)
print(coord2city(test))
print(coord2city(rand_coord()))
coord2city(rand_coord())[0]

test = rand_city_data()
test.append(rand_city_data(),ignore_index = True)

[-51.11, 33.95]
['port alfred', 'za']
['hobart', 'au']


,City,Country Code,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,URL
0,atuona,pf,1.52,-150.09,79.55,100,20,11.77,http://api.openweathermap.org/data/2.5/weather...
1,busselton,au,-88.23,87.51,72.04,88,0,17.58,http://api.openweathermap.org/data/2.5/weather...
